In [1]:
import re


In [2]:
file1 = open('input.txt', 'r')
lines = file1.readlines()


In [3]:
seeds = []
seed_groups = []
maps = {}
current_map = ()
for line in lines:
    if re.match(r'seeds:', line) is not None:
        seeds_output = re.findall(r'\b\d+\b', line)
        seeds = seeds_output
        for i in range(0, len(seeds_output), 2):
            seed_groups.append((int(seeds_output[i]), int(seeds_output[i+1])))
    if line == "\n" or not line:
        continue

    if re.match(r'(\w*)-to-(\w*) map', line) is not None:
        current_map =re.match(r'(\w*)-to-(\w*) map', line).groups()
        maps[f"{current_map[0]}-to-{current_map[1]}"] = {}
        maps[f"{current_map[1]}-to-{current_map[0]}"] = {}
        continue

    if re.match(r'^\d+ \d+ \d+$', line) is not None:
        (elem1, elem2, cnt) = re.match(r'^(\d+) (\d+) (\d+)$', line).groups()
        maps[f"{current_map[0]}-to-{current_map[1]}"][int(elem2)] = {"diff": int(elem1) - int(elem2), "cnt": int(cnt)}
        maps[f"{current_map[1]}-to-{current_map[0]}"][int(elem1)] = {"diff": int(elem2) - int(elem1), "cnt": int(cnt)}


In [4]:
def get_value(dict, key):
    key_under = max([i for i in list(dict.keys()) if i <= key], default=None)
    if key_under is None:
        return key
    if dict[key_under]["cnt"] + key_under < key:
        return key
    return key+dict[key_under]["diff"]


In [5]:
min_value = 999999999999999999999999999999999999
seed_details = {}

for seed in seeds:
    soil = get_value(maps["seed-to-soil"], int(seed))
    fertilizer = get_value(maps["soil-to-fertilizer"], soil)
    water = get_value(maps["fertilizer-to-water"], fertilizer)
    light = get_value(maps["water-to-light"], water)
    temperature = get_value(maps["light-to-temperature"], light)
    humidity = get_value(maps["temperature-to-humidity"], temperature)
    location = get_value(maps["humidity-to-location"], humidity)
    if location < min_value:
        min_value = location
    seed_details[seed] = {
        "seed": seed,
        "soil": soil,
        "fertilizer": fertilizer,
        "water": water,
        "light": light,
        "temperature": temperature,
        "humidity": humidity,
        "location": location
    }
print(f"part1: {min_value}")


part1: 403695602


In [9]:
def get_value_range(dict, ranges, output_ranges = []):
    if len(ranges) == 0:
        return output_ranges
    key_under = max([i for i in list(dict.keys()) if i <= ranges[0][0]], default=None)
    if key_under is None:
        return get_value_range(dict, ranges[1:], output_ranges + [(ranges[0][0], ranges[0][1])])
    range_length = dict[key_under]["cnt"]
    if key_under <= ranges[0][0] < range_length + key_under: # if we start
        if ranges[0][0]+ranges[0][1] <= range_length + key_under: #fullRange is in the first range\
            return get_value_range(dict, ranges[1:], output_ranges + [(ranges[0][0] + dict[key_under]["diff"], ranges[0][1])])
        else:
            rr = ranges[1:] + [(ranges[0][0] + ( range_length - (ranges[0][0]-key_under)), ranges[0][1] - ( range_length - (ranges[0][0]-key_under)))]
            rro = output_ranges + [(ranges[0][0] + dict[key_under]["diff"], range_length - (ranges[0][0]-key_under))]
            return get_value_range(dict, rr, rro)
    else:
        return get_value_range(dict, ranges[1:], output_ranges + [(ranges[0][0], ranges[0][1])])


In [11]:
import time

# get the start time
st = time.time()
final_location = []
for seed_tupple in seed_groups:
    soil = get_value_range(maps["seed-to-soil"], [seed_tupple])
    fertilizer = get_value_range(maps["soil-to-fertilizer"], soil)
    water = get_value_range(maps["fertilizer-to-water"], fertilizer)
    light = get_value_range(maps["water-to-light"], water)
    temperature = get_value_range(maps["light-to-temperature"], light)
    humidity = get_value_range(maps["temperature-to-humidity"], temperature)
    location = get_value_range(maps["humidity-to-location"], humidity)
    final_location.append(location)

min_value = 9999999999999
for location in final_location:
    for points in location:
        if points[0] < min_value:
            min_value = points[0]
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')
print(f"part2: {min_value}")


TypeError: object of type 'NoneType' has no len()